# Ensemble Learning - 2
***

지난 챕터에서는 앙상블 알고리즘 중 하나인 RandomForest에 대해 알아보았는데요. 이번 챕터에서는 부스팅 계열의 알고리즘에 대해 살펴보도록 하겠습니다. 먼저 부스팅 알고리즘은 여러 개의 약한 학습기(weak learner)를 학습 및 예측해가면서 잘못 예측된 데이터에 대해 가중치를 부여하여 오류를 개선해가면서 학습하는 방식입니다. 

대표적인 부스팅 AdaBoost와 GradientBoost가 있습니다. 

먼저 AdaBoost는 Adaptive + Boosting으로 이전 분류기가 잘못 예측한 데이터의 가중치를 adaptive하게 수정해가며 잘못 예측된 데이터에 가중치를 부여하고 더 집중하여 오류를 개선해나갑니다.

반면 GradientBoost는 AdaBoost와 유사하지만 가중치 업데이트를 경사하강법(Gradient Descent)을 활용합니다. 실제 값과 예측 값의 차이를 최소로 줄일 수 있는 방향으로 반복적으로 가중치를 업데이트 하는 방식을 통해 데이터를 학습하고 예측하게 됩니다.
***
바로 실습을 통해서 다른 모델들과 비교를 해가면서 자세히 들여다 보겠습니다

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
cancer = load_breast_cancer()

In [3]:
log_clf = LogisticRegression() # 로지스틱 회귀 모델
ada_clf = AdaBoostClassifier(random_state = 1212) # 에이다부스트 모델
gbm_clf = GradientBoostingClassifier(random_state = 1212) # 그래디언트부스트 모델

In [4]:
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size = .2, random_state = 1212)

In [5]:
log_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)
gbm_clf.fit(X_train, y_train)

GradientBoostingClassifier(random_state=1212)

In [6]:
log_acc = np.round(accuracy_score(y_test,log_clf.predict(X_test)), 6) * 100
ada_acc = np.round(accuracy_score(y_test, ada_clf.predict(X_test)), 4) * 100
gbm_acc = np.round(accuracy_score(y_test, gbm_clf.predict(X_test)), 4) * 100

In [7]:
print(f'{log_clf.__class__.__name__}의 정확도는 {log_acc}%')
print(f'{ada_clf.__class__.__name__}의 정확도는 {ada_acc}%')
print(f'{gbm_clf.__class__.__name__}의 정확도는 {gbm_acc}%')

LogisticRegression의 정확도는 94.7368%
AdaBoostClassifier의 정확도는 96.49%
GradientBoostingClassifier의 정확도는 95.61%


기본 로지스틱 회귀 모델 보다 성능이 좋은 것을 확인하였습니다. 

부스팅 계열도 트리 기반의 알고리즘이기 때문에 하이퍼 파라미터는 트리 알고리즘이 가지는 하이퍼 파라미터는 기본적으로 가지고 있습니다.

In [8]:
from sklearn.model_selection import GridSearchCV

In [9]:
param = {'n_estimators' : [20, 30, 50, 100 ,300 ,500], 'max_depth' : [3,4,5,6,7], 
        'learning_rate' : [0.005, 0.01, 0.05, 0.1, 0.2]}

In [10]:
gbm_clf = GradientBoostingClassifier(random_state = 1212)

grid_gbm = GridSearchCV(gbm_clf, param, n_jobs = -1)

In [11]:
grid_gbm.fit(X_train, y_train)

GridSearchCV(estimator=GradientBoostingClassifier(random_state=1212), n_jobs=-1,
             param_grid={'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.2],
                         'max_depth': [3, 4, 5, 6, 7],
                         'n_estimators': [20, 30, 50, 100, 300, 500]})

In [12]:
print('최적의 파라미터 조합은',grid_gbm.best_params_)
print(f'최고 정확도는 {np.round(grid_gbm.best_score_, 4) * 100}%')

최적의 파라미터 조합은 {'learning_rate': 0.2, 'max_depth': 4, 'n_estimators': 100}
최고 정확도는 96.92%


해당 파라미터를 가지고 최종 예측을 해보도록 하겠습니다.

In [13]:
final_gbm_clf = GradientBoostingClassifier(**grid_gbm.best_params_, random_state = 1212)
final_gbm_clf.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.2, max_depth=4, random_state=1212)

In [14]:
final_acc = np.round(accuracy_score(y_test, final_gbm_clf.predict(X_test)), 4) * 100

In [15]:
print(f'최종 {final_gbm_clf.__class__.__name__}의 정확도는 {final_acc}%')

최종 GradientBoostingClassifier의 정확도는 97.37%


GradientBoosting 모델은 과적합에도 덜 취약한 알고리즘이지만 수행 시간이 오래 걸린다는 단점이 있는데요. 이러한 단점을 보완하고 현재 ML에서 제일 널리 사용되고 있는 XGBoost와 LightGBM에 대해서 알아보도록 하겠습니다.
***

XGBoost(eXtra Gradient Boost)는 아래와 같은 장점을 가지고 있습니다.

 
| 장점 | 설명 |
|:--------|:--------:|
| 뛰어난 성능  | 분류와 회귀에서 뛰어난 성능을 보여준다. |
| 빠른 수행 시간 | GradientBoosting의 느린 수행 시간 보완 |
| 과적합 규제 | 과적합에 좀 더 강함 |
| 나무 가지치기 | 가지치기로 분할 수를 감소 시킨다. |
| 자체 교차 검증 가능 | 교차 검증 및 조기 중단 기능 내장 |
| 결측치 자체 처리 | 자체적으로 결측치 처리 가능 |

***
XGBoost는 XGBoost API를 사용하는 방법과 sklearn 전용 방법 두 가지가 있는데 XGBoost API를 먼저 사용해보도록하겠습니다.

In [16]:
import xgboost as xgb

XGBoost는 전용 API 사용시 DMatrix 객체를 생성하여 사용합니다. DMatrix는 Numpy 배열을 입력으로 받아 만들어지는 XGBoost 전용 데이터 포맷입니다. 

In [19]:
dtrain = xgb.DMatrix(data = X_train, label = y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)

데이터를 구성한 후 train() 메서드를 사용하여 학습을 진행합니다. XGBoost에는 조기중단 기능이 제공되는데 조기 중단 기능은 학습시에 성능이 개선되지 않을 경우 학습을 종료하는 기능입니다. 

In [47]:
xgb_clf = xgb.train(params = {'max_depth' : 4}, dtrain = dtrain, num_boost_round = 100, evals = [(dtrain, 'train'), (dtest, 'test')])

[0]	train-rmse:0.36225	test-rmse:0.38009
[1]	train-rmse:0.26756	test-rmse:0.30057
[2]	train-rmse:0.19704	test-rmse:0.25099
[3]	train-rmse:0.14981	test-rmse:0.22035
[4]	train-rmse:0.11599	test-rmse:0.20590
[5]	train-rmse:0.09390	test-rmse:0.20095
[6]	train-rmse:0.07849	test-rmse:0.19697
[7]	train-rmse:0.06989	test-rmse:0.19453
[8]	train-rmse:0.06111	test-rmse:0.19250
[9]	train-rmse:0.05696	test-rmse:0.19268
[10]	train-rmse:0.04995	test-rmse:0.19177
[11]	train-rmse:0.04598	test-rmse:0.19176
[12]	train-rmse:0.04297	test-rmse:0.19165
[13]	train-rmse:0.03933	test-rmse:0.19168
[14]	train-rmse:0.03500	test-rmse:0.19177
[15]	train-rmse:0.03156	test-rmse:0.19168
[16]	train-rmse:0.03022	test-rmse:0.19172
[17]	train-rmse:0.02799	test-rmse:0.19195
[18]	train-rmse:0.02698	test-rmse:0.19204
[19]	train-rmse:0.02519	test-rmse:0.19105
[20]	train-rmse:0.02477	test-rmse:0.19113
[21]	train-rmse:0.02440	test-rmse:0.19110
[22]	train-rmse:0.02268	test-rmse:0.19103
[23]	train-rmse:0.02137	test-rmse:0.19106
[2

In [48]:
pred = xgb_clf.predict(dtest)

XGBoost의 API는 predict 메서드 실행시 확률 값을 반환하게 됩니다.

In [49]:
pred[:10] # 반환된 확률 10개 추출

array([ 1.0031145 ,  1.0087953 ,  0.7946443 ,  1.0017257 ,  1.0014106 ,
        0.9946285 ,  0.99687135,  0.99919975,  0.9931866 , -0.01643419],
      dtype=float32)

In [50]:
pred = [1 if x > .5 else 0 for x in pred] # 확률 값을 0.5 를 기준으로 binary 형태로 변환

In [51]:
pred[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]

In [52]:
accuracy_score(y_test, pred)

0.9649122807017544

XGBoost API 활용시 자체적으로 교차 검증(cross validation)이 가능한데요. xgb.cv()를 활용하여 자체적으로 교차 검증이 가능합니다.

In [79]:
cv_param = {'max_depth' : 6, 'eta' : 0.005, 'objective' : 'binary:logistic'}

In [80]:
cv_xgb = xgb.cv(params = cv_param, dtrain = dtrain, num_boost_round = 100,
               nfold = 5, stratified = True, metrics = 'error')

***
sklearn의 XGBoost 클래스를 사용하여 작업을 진행할 수 있습니다.

In [84]:
from xgboost import XGBClassifier

In [88]:
xgb_clf = XGBClassifier(random_state = 1214, n_estimators = 100,max_depth = 4) # XGBppst API의 'num_boost_round'와 'n_estimators'는 같은 파라미터

In [89]:
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              random_state=1214, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [90]:
pred = xgb_clf.predict(X_test)

In [92]:
print(accuracy_score(y_test, pred)) # 위의 XGBoost API 사용 결과와 동일하다.

0.9649122807017544


sklearn의 XGBoostClassifier도 조기 중단 기능을 사용할 수 있습니다. 

In [95]:
xgb_clf.fit(X_train, y_train, early_stopping_rounds = 100, eval_metric = 'logloss', eval_set = [(X_test, y_test)], verbose=0) # verbose = 0이면 과정 출력 생략

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              random_state=1214, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [96]:
pred = xgb_clf.predict(X_test)

In [97]:
print(accuracy_score(y_test, pred))

0.9649122807017544


#### <font color = 'red'>주의!
    
현재 챕터에서는 모델 학습시 예측해야할 X_test도 학습과정시 검증용 데이터로 사용하였습니다
ex> fit(~~~, eval_set = [(X_test, y_test)])
하지만 실제 문제 해결시에는 우리가 예측해야할 데이터는 모델링 진행시 절대로 사용하면 안됩니다.
    
해당 문제는 data leakage의 문제가 있기 때문에 예측해야할 데이터를 사전에 학습하는 누수의 문제점이 있습니다.
    
### <font color = 'blue'> 따라서 train 데이터만 모델 학습에 사용하고 최종 예측해야할 데이터는 모델링 진행시 사용하시면 안됩니다!
    
*** 
지금까지 앙상블 알고리즘 중에서 XGBoost에 대해서 알아보았습니다. 다음 챕터에서는 XGBoost 처럼 ML에서 각광을 받고 있는 Lightgbm에 대해 살펴보도록 하겠습니다. 감사합니다.